In [1]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
import math

In [2]:
def knn_with_distance_list(data, k):
    rows_count = len(data)
    
    knn_list = []
    for i in range(rows_count):
        knn_list.append([])
        
    nbrs = NearestNeighbors(n_neighbors=k, algorithm='ball_tree').fit(data)
    distances, indices = nbrs.kneighbors() 

    for index, knnindex, distindex in list(zip(range(len(indices)), indices, distances)):
        for k, d in list(zip(knnindex, distindex)):
            knn_list[index].append((k, d))
                
    return knn_list

# 计算法向量
def resultant_force(data, knn_list):
    resultant_force_list = []
    k_norm_vec = []
    local_dense_list = []
        
    for j in range(len(data)):
        weight_sum = 0
        local_dense = 0
        vector_sum = np.zeros(len(data[0]))
        norm_vec = []
        
        for index, dist in knn_list[j]:
            # 计算局部密度
            dis_index = knn_list[index][len(knn_list[0])-1][1]
            real_dist = max(dis_index, dist)
            local_dense += real_dist     
            # 计算合力
            if dist != 0:
                vector = (data[index] - data[j])/dist
                norm_vec.append(vector)
                vector_sum += vector

        local_dense = 1 / local_dense
        local_dense_list.append(local_dense)
        k_norm_vec.append(norm_vec)
        resultant_force = vector_sum
        resultant_force_list.append(resultant_force)
            
    return (resultant_force_list, k_norm_vec, local_dense_list)

# 计算LOF
def LOF(knn_list, k, local_dense_list):
    rows_count = len(local_dense_list)
    LOF_list = []
    
    for i in range(rows_count):
        LOF = 0
        for index, dist in knn_list[i]:
            LOF += (local_dense_list[index] / local_dense_list[i])
    
        LOF = LOF / k
        LOF_list.append(LOF)
    
    LOF_arr = np.array(LOF_list)
    max_LOF = np.max(LOF_arr)
    min_LOF = np.min(LOF_arr)
    if (max_LOF - min_LOF) > 0:
        LOF_arr = (LOF_arr - min_LOF) / (max_LOF - min_LOF)
        
    return LOF_arr

# 计算超切平面度量及综合三个子度量
def tangent_plane(resultant_force_list, k_norm_vec, k, LOF_arr, ratio):
    num = len(resultant_force_list)
    diff_var_list = []
    p_points_ratio_arr = np.zeros(num)
    ind = np.zeros(num)
    for i in range(num):
        p_eps = 0
        n_eps = 0
        p_vec = []
        n_vec = []
        p_var = []
        n_var = []
        for j in range(len(k_norm_vec[i])):
            dot = np.dot(k_norm_vec[i][j], resultant_force_list[i])
            if dot>=0:
                p_eps += 1
                p_vec.append(k_norm_vec[i][j])
            else:
                n_eps += 1
                n_vec.append(k_norm_vec[i][j])
        
        p_points_ratio_arr[i] = p_eps/k
        if (n_eps > 1 and p_eps > 1):
            for ind1 in range(len(p_vec) - 1):
                for ind2 in range(ind1 + 1, len(p_vec)):
                    p_var.append(np.dot(p_vec[ind1], p_vec[ind2]))
            
            for ind1 in range(len(n_vec) - 1):
                for ind2 in range(ind1 + 1, len(n_vec)):
                    n_var.append(np.dot(n_vec[ind1], n_vec[ind2]))
            diff_var = abs(np.var(p_var) - np.var(n_var))
            diff_var_list.append(diff_var)
        else:
            diff_var_list.append(0)
    
    diff_var_arr = np.array(diff_var_list)
    max_var = np.max(diff_var_arr)
    diff_var_arr[diff_var_arr==0] = max_var
            
    min_var = np.min(diff_var_arr)
    if(max_var - min_var > 0):
        diff_var_arr = (diff_var_arr - min_var) / (max_var - min_var)
        
    BD_arr = 0.6*p_points_ratio_arr + 0.2*diff_var_arr + 0.2*LOF_arr
    BD_arr_sort = abs(np.sort(-BD_arr))
    order = math.ceil(ratio*num) - 1
    threshold = BD_arr_sort[order]
    for i in range(num):
        if BD_arr[i] >= threshold:
            ind[i] = 1
    
    return ind
        
def boundary_detection(data, k, ratio):
    k = min(k, len(data) - 1)
    knn_with_dist = knn_with_distance_list(data, k)
    resultant_force_all = resultant_force(data, knn_with_dist)
    LOF_list = LOF(knn_with_dist, k, resultant_force_all[2])
    boundary_result = tangent_plane(resultant_force_all[0], resultant_force_all[1], k, LOF_list, ratio)
    
    return boundary_result
